#### This is used to go in the same directory as when everything is run under Pycharm in a normal python file

In [1]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\Fede\Documents\GitHub\RelGAN\src\jupyter_tests


In [2]:
os.chdir("..")
cwd = os.getcwd()
print(cwd)
assert cwd.endswith("src")

C:\Users\Fede\Documents\GitHub\RelGAN\src


# Test on KL divergence

In [35]:
from topic_modelling.lda_topic import train_specific_LDA
from topic_modelling.lda_utils import get_corpus
from path_resolution import resources_path
from real.real_gan.real_loader import RealDataTopicLoader
from utils.text_process import text_precess
import random
import numpy as np
import gc
from scipy.stats import entropy

In [4]:
data_file = resources_path("data", '{}.txt'.format("image_coco"))
coco = True
corpus_raw = get_corpus(coco, datapath=data_file)
lda = train_specific_LDA(corpus_raw, num_top=42, passes=2, iterations=2, chunksize=2000, coco=coco)

No model found
New model saved in 75.564199924469 sec


In [8]:
data_file = resources_path("data", '{}.txt'.format("image_coco"))
oracle_file = resources_path(os.path.join('.', 'oracle', 'samples'), '{}.txt'.format("image_coco"))
topic_number = 42

seq_len, vocab_size, word_index_dict, index_word_dict = text_precess(data_file, oracle_file=oracle_file)
oracle_loader = RealDataTopicLoader(64, seq_len)
oracle_loader.topic_num = topic_number
oracle_loader.get_LDA(word_index_dict, index_word_dict, data_file)

Computation of topic model started...
No model found
New model saved in 72.5709011554718 sec
number of LDA words: 3499
Topic model computed in 73.84313559532166 sec!


In [61]:
def computeKL(corpus_raw, data_loader):
    s = random.sample(corpus_raw, 64)
    #real_sentences, generated_sentences = get_sentences()
    #real_sentences_topic = data_loader.get_topic(real_sentences)
    real_sentences_topic = data_loader.get_topic(s)
    #generated_sentences_topic = data_loader.get_topic(generated_sentences)
    mean_topic = get_avg_topic(oracle_loader).squeeze()
    result = []
    for real in real_sentences_topic:
        #print("Sum: {}".format(np.sum(real)))
        real = real / np.sum(real)
        #print("Sum2: {}, shape: {}".format(np.sum(real), real.shape))
        #print("Mean topic sum: {}, shape: {}".format(np.sum(mean_topic), mean_topic.shape))
        r = entropy(list(real), list(mean_topic))
        s = entropy(mean_topic, real)
        print(r, s)
        result.append(r)

    return np.mean(result)

def get_avg_topic(oracle_loader):
    topic_matrix = oracle_loader.lda.lda_model.get_topics()
    topic_weights = np.ones(oracle_loader.lda.topic_num) / oracle_loader.lda.topic_num
    topic_weights = np.expand_dims(topic_weights, 0)
    topic_sentences = np.dot(topic_weights, topic_matrix)  # num_sentences x num_word
    topic_sentences = np.divide(topic_sentences,
                                np.sum(topic_sentences, axis=1, keepdims=True))  # rowwise normalization

    real_vector = np.zeros(
        (topic_sentences.shape[0], len(oracle_loader.model_word_index_dict) + 1))  # sentence_number x vocab_size

    for ind, invere_index in enumerate(oracle_loader.inverse_indexes):
        # more than one index in the model because of lemmatization
        for x in invere_index:
            real_vector[:, x] = topic_sentences[:, ind]

    gc.collect()
    real_vector = np.divide(real_vector, np.sum(real_vector, axis=1, keepdims=True))
    return real_vector

In [62]:
computeKL(corpus_raw, oracle_loader)

1.931765130427534 2.524536939460581
1.1767796582351129 1.701997476006727
1.408009819399247 1.9648417565746599
1.661483605415608 2.1895545321152636
1.6043477526406984 2.152708097571867
1.1897241651413262 1.6948845442474347
1.533888031843519 1.9094607466299667
1.8433247874510381 2.3130325577015136
1.3475960827230529 1.9408383620825984
1.450714881254403 1.8867700796021007
1.4022009632930774 1.8449297335523427
1.1699299534071352 1.6142156913322305
2.0096761331991386 2.4614745472473993
1.5225184734692747 2.196596366939604
1.5875365628829756 2.397723546671622
2.0478505680363988 2.464883418542761
1.4984258403278325 1.8065387906572197
1.6163465677658706 1.9657427275229378
1.8450388381803746 2.2696866071796595
1.516854489474884 2.3697282505345414
1.7834908632633695 2.0783918835740183
1.4791112573580567 1.7349884000363949
1.16844908369571 1.656533472999754
1.7066010644203546 2.121411969612905
1.8126570810229257 2.642172573344153
1.7032343329344817 1.9967950343273373
1.178280142317364 1.691699291

1.599142494714898

In [43]:
mean_topic = get_avg_topic(oracle_loader).squeeze()

In [71]:
s = random.sample(corpus_raw, 64)
#real_sentences, generated_sentences = get_sentences()
#real_sentences_topic = data_loader.get_topic(real_sentences)
real_sentences_topic = oracle_loader.get_topic(s)
real_sentences_topic = np.divide(real_sentences_topic, np.sum(real_sentences_topic, axis=1, keepdims=True))
#generated_sentences_topic = data_loader.get_topic(generated_sentences)
mean_topic = get_avg_topic(oracle_loader).squeeze()

In [72]:
np.sum(real_sentences_topic, axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [44]:
result = []
for real in real_sentences_topic:
    #print("Sum: {}".format(np.sum(real)))
    real = real / np.sum(real)
    #print("Sum2: {}, shape: {}".format(np.sum(real), real.shape))
    #print("Mean topic sum: {}, shape: {}".format(np.sum(mean_topic), mean_topic.shape))
    r = entropy(list(real), list(mean_topic))
    s = entropy(mean_topic, real)
    print(r, s)
    result.append(r)

0.9999999999999999

In [82]:
np.sum((real_sentences_topic[0] - mean_topic) ** 2)

0.015012192635322948

In [83]:
np.mean(real_sentences_topic[0])

0.0002135839384878257